In [ ]:
import json
import subprocess
import typing

from dataclasses import dataclass
from typing import List, Literal, Union, Annotated

import cattrs

In [ ]:
subprocess.run(["cargo", "build", "-p", "io-patterns", "--release"], check=True)

# Input / output

In [ ]:
@dataclass
class Input:
    value: int


@dataclass
class Output:
    value: int

In [ ]:
inp = Input(value=21)
inp = cattrs.unstructure(inp)
inp = json.dumps(inp)

res = subprocess.run(
    ["./target/release/io-patterns-double.exe"],
    input=inp,
    encoding="utf-8",
    capture_output=True,
    check=True,
)

out = json.loads(res.stdout)
out = cattrs.structure(out, Output)
out

In [ ]:
def call(executable, inp, ty, converter=cattrs):
    inp = cattrs.unstructure(inp)
    inp = json.dumps(inp)

    res = subprocess.run(
        [str(executable)],
        input=inp,
        encoding="utf-8",
        capture_output=True,
        check=True,
    )

    out = json.loads(res.stdout)
    return cattrs.structure(out, ty)

In [ ]:
call("./target/release/io-patterns-double.exe", Input(value=42), Output)

# Streaming IO

In [ ]:
inputs = ["hello", "world", "from", "python"]
Output = str


def call_streaming(path, inputs, output_type):
    with subprocess.Popen(
        [path],
        encoding="utf-8",
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
    ) as proc:
        for inp in inputs:
            inp = json.dumps(cattrs.unstructure(inp))

            proc.stdin.write(inp)
            proc.stdin.write("\n")
            proc.stdin.flush()

            out = proc.stdout.readline().rstrip()

            yield cattrs.structure(json.loads(out), output_type)

        proc.stdin.close()

    assert proc.returncode == 0


for out in call_streaming(
    "./target/release/io-patterns-echo.exe",
    inputs,
    str,
):
    print(out)

# Working with unions

In [ ]:
@dataclass
class Input:
    floats: List[float]
    flexible: Union[str, float]


@dataclass
class Output:
    sum: float
    flexible: Union[str, float]

In [ ]:
def structure_untagged_union(obj, ty):
    assert typing.get_origin(ty) is typing.Union

    errors = []
    for cand_ty in typing.get_args(ty):
        try:
            return cattrs.structure(obj, cand_ty)

        except Exception as exc:
            errors.append(exc)

    raise ValueError(f"Cannot struct object as {ty}: {errors}")


# register a custom structure hook
cattrs.register_structure_hook(Union[str, float], structure_untagged_union)

In [ ]:
call(
    "./target/release/io-patterns-complex.exe",
    Input(
        floats=[1.0, 2.0, 3.0, 4.0],
        flexible="foo",
    ),
    Output,
)

## More unions for cattrs

### External tags

Serialize unions of the from:

```json
{
    {type}: {value}
}
```

For example:

```json
{"str": "hello world"}
{"float": 123.0}
```

In [ ]:
TaggedOutput = Annotated[Union[str, float], "external-tag"]

In [ ]:
def supports_structure_external_tag(ty):
    if typing.get_origin(ty) is not Annotated:
        return False

    base_ty, *annotations = typing.get_args(ty)

    if typing.get_origin(base_ty) is not typing.Union:
        return False

    if "external-tag" not in annotations:
        return False

    return all(hasattr(arg, "__name__") for arg in typing.get_args(base_ty))


def structure_external_tag(obj, ty):
    assert typing.get_origin(ty) is Annotated
    assert isinstance(obj, dict) and len(obj) == 1

    base_ty, *_ = typing.get_args(ty)
    assert typing.get_origin(base_ty) is typing.Union

    types = {child.__name__: child for child in typing.get_args(base_ty)}

    ((key, val),) = obj.items()
    return cattrs.structure(val, types[key])

In [ ]:
cattrs.register_structure_hook_func(
    supports_structure_external_tag,
    structure_external_tag,
)

In [ ]:
call("./target/release/io-patterns-complex-tagged.exe", "string", TaggedOutput)

### Internally tag unions

Serialize unions with a `type` tag:

```json
{
    "type": "...",
    "...",
}
```

In [ ]:
def supports_structure_type_tag(ty, *, type_tag="type"):
    if typing.get_origin(ty) is not typing.Union:
        return False

    for child in typing.get_args(ty):
        hints = typing.get_type_hints(child)
        if "type" not in hints:
            return False

        type_hint = hints[type_tag]
        if typing.get_origin(type_hint) is not typing.Literal:
            return False

        type_args = typing.get_args(type_hint)
        if len(type_args) != 1 or not isinstance(type_args[0], str):
            return False

    return True


def structure_type_tag(obj, ty, *, type_tag="type", converter=cattrs):
    assert isinstance(obj, dict)
    type_tag_value = obj[type_tag]

    for child in typing.get_args(ty):
        hints = typing.get_type_hints(child)
        (type_tag_value_child,) = typing.get_args(hints[type_tag])
        if type_tag_value == type_tag_value_child:
            return child(obj)

    raise ValueError(f"Cannot structure {obj}")

In [ ]:
cattrs.register_structure_hook_func(supports_structure_type_tag, structure_type_tag)

In [ ]:
@dataclass
class TaggedFloat:
    value: float
    type: Literal["float"] = "float"


@dataclass
class TaggedString:
    value: str
    type: Literal["str"] = "str"


print(
    cattrs.structure({"type": "float", "value": 42}, Union[TaggedFloat, TaggedString])
)
print(
    cattrs.structure({"type": "str", "value": "foo"}, Union[TaggedFloat, TaggedString])
)